In [1]:
import pandas as pd
import numpy as np
import keras
import scipy
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from lineartree import RandomForestRegressor
from sklearn import metrics
from scipy.signal import savgol_filter
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [2]:
from numpy.random import seed
seed(1)
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [3]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [4]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [5]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [6]:
def predict_dist(X, model, num_samples):
    preds = [model(X, training=True) for _ in range(num_samples)]
    return np.hstack(preds)

In [7]:
def predict_point(X, model, num_samples):
    pred_dist = predict_dist(X, model, num_samples)
    return pred_dist.mean(axis=1)

In [8]:
#Define models

initializer = keras.initializers.HeNormal(seed=1)
opt = keras.optimizers.Adam(learning_rate=0.001)

model_peak = keras.models.Sequential()
model_peak.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_peak.compile(loss='mean_squared_error', optimizer= opt)

model_fwhm = keras.models.Sequential()
model_fwhm.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_fwhm.compile(loss='mean_squared_error', optimizer= opt)

model_cascade = keras.models.Sequential()
model_cascade.add(Dense(6, input_shape=(6,), kernel_initializer=initializer, activation='relu'))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_cascade.compile(loss='mean_squared_error', optimizer= opt)


In [9]:
#Xval on Train/Val set, 10-fold cross validation (Group splits), PL Peak, FWHM (eV), Normalized
# load dataset into Pandas DataFrame (cleaned PL data)
df = pd.read_csv('CdSe PL Train Val.csv')

#normalize input and output data
min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)

X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
X2 = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s','Peak_eV']]
y = df['Peak_eV']
z = df['FWHM_eV']
group = df['Expt. No']

#Split groups for cross validation by Experiment number
gkf = GroupKFold(n_splits=10)
matrix = np.zeros([10,16])
matrix_fwhm = np.zeros([10,16])
matrix_cascade = np.zeros([10,16])

j = 0
#Loop through the 10 splits
for train_index, test_index in gkf.split(X, groups=group):
    #Separate out training and test sets
    X_train = X.iloc[train_index, :]
    X2_train = X2.iloc[train_index, :]
    y_train = y[train_index]
    z_train = z[train_index]
    X_test = X.iloc[test_index, :]
    X2_test = X2.iloc[test_index, :]
    y_test = y[test_index]
    z_test = z[test_index]
    
    #Load weights
    model_peak.load_weights('initialize_weights3.h5')
    model_fwhm.load_weights('initialize_weights3.h5')
    model_cascade.load_weights('initialize_weights4.h5')

    #Train models
    #prediction for Peak
    model_peak.fit(X_train, y_train, epochs=300, validation_data=(X_test, y_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred = model_peak.predict(X_test)
    #prediction for FWHM (direct)
    model_fwhm.fit(X_train, z_train, epochs=300, validation_data=(X_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred_fwhm = model_fwhm.predict(X_test)
    #prediction for FWHM (cascade)
    model_cascade.fit(X2_train, z_train, epochs=300, validation_data=(X2_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    X_test.insert(5, "Peak_eV", model_peak.predict(X_test), True)
    y_pred_cascade = model_cascade.predict(X_test)

    
    #prediction for Peak with dropout
    X_test = X.iloc[test_index, :]
    X_test_t = K.constant(X_test)
    y_pred_d =  predict_point(X_test_t, model_peak, 1000)
    #prediction for FWHM (direct)
    y_pred_fwhm_d = predict_point(X_test_t, model_fwhm, 1000)
    #prediction for FWHM (cascade)
    X_test.insert(5, "Peak_eV", y_pred_d, True)
    X_test_t2 = K.constant(X_test)
    y_pred_cascade_d = predict_point(X_test_t2, model_cascade, 1000)
    
    #errors in eV (normalized)
    #UN-normalize eV predictions by scaling with ratio

    ratio = (1239.84193/400 - 1239.84193/620)
    ratio2 = 0.35
    matrix[j,0] = metrics.mean_absolute_error(y_test, y_pred)*ratio
    matrix[j,1] = metrics.r2_score(y_test, y_pred)
    matrix[j,2] = rae(y_test, y_pred)
    matrix[j,3] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    matrix[j,4] = metrics.mean_absolute_error(y_test, y_pred_d)*ratio 
    matrix[j,5] = metrics.r2_score(y_test, y_pred_d)
    matrix[j,6] = rae(y_test, y_pred_d)
    matrix[j,7] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred_d))

    matrix_fwhm[j,0] = metrics.mean_absolute_error(z_test, y_pred_fwhm)*ratio2
    matrix_fwhm[j,1] = metrics.r2_score(z_test, y_pred_fwhm)
    matrix_fwhm[j,2] = rae(z_test, y_pred_fwhm)
    matrix_fwhm[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm))
    matrix_fwhm[j,4] = metrics.mean_absolute_error(z_test, y_pred_fwhm_d)*ratio2
    matrix_fwhm[j,5] = metrics.r2_score(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,6] = rae(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm_d))
    
    matrix_cascade[j,0] = metrics.mean_absolute_error(z_test, y_pred_cascade)*ratio2
    matrix_cascade[j,1] = metrics.r2_score(z_test, y_pred_cascade)
    matrix_cascade[j,2] = rae(z_test, y_pred_cascade)
    matrix_cascade[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade))
    matrix_cascade[j,4] = metrics.mean_absolute_error(z_test, y_pred_cascade_d)*ratio2
    matrix_cascade[j,5] = metrics.r2_score(z_test, y_pred_cascade_d)
    matrix_cascade[j,6] = rae(z_test, y_pred_cascade_d)
    matrix_cascade[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade_d))
    
    

    #errors in nm
    #UNnormalize the data first
    y_test = y_test*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred = y_pred*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred_d = y_pred_d*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    z_test = z_test*(0.35)+0.05
    y_pred_fwhm = y_pred_fwhm*(0.35)+0.05
    y_pred_fwhm_d = y_pred_fwhm_d*(0.35)+0.05
    y_pred_cascade = y_pred_cascade*(0.35)+0.05
    y_pred_cascade_d = y_pred_cascade_d*(0.35)+0.05
    
    #Calculate data in nm and eV
    matrix[j,8] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,9] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,10] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,11] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
    matrix[j,12] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,13] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,14] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,15] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d)))

    matrix_fwhm[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm)))
    matrix_fwhm[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))
    matrix_fwhm[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))) 

    matrix_cascade[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade)))
    matrix_cascade[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d)))

    j+=1

matrix_2 = np.zeros([1,32])
matrix_2[0,:16] = matrix.mean(0)[:16]
matrix_2[0,16:32] = np.std(matrix[:,:16], axis=0)

matrix_3 = np.zeros([1,32])
matrix_3[0,:16] = matrix_fwhm.mean(0)[:16]
matrix_3[0,16:32]= np.std(matrix_fwhm[:,:16], axis=0)

matrix_4 = np.zeros([1,32])
matrix_4[0,:16] = matrix_cascade.mean(0)[:16]
matrix_4[0,16:32] = np.std(matrix_cascade[:,:16], axis=0)

np.set_printoptions(precision=5, threshold=5, edgeitems=4, suppress=True)


Epoch 1/300
36/36 [==============================] - 1s 5ms/step - loss: 0.0642 - val_loss: 0.0390
Epoch 2/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0283 - val_loss: 0.0175
Epoch 3/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0188 - val_loss: 0.0100
Epoch 4/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0137 - val_loss: 0.0077
Epoch 5/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0134 - val_loss: 0.0069
Epoch 6/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0112 - val_loss: 0.0063
Epoch 7/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0101 - val_loss: 0.0059
Epoch 8/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0092 - val_loss: 0.0052
Epoch 9/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0083 - val_loss: 0.0053
Epoch 10/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0081 - val_loss: 0.0052
Epoch 11/

36/36 [==============================] - 0s 2ms/step - loss: 0.0029 - val_loss: 0.0027
Epoch 84/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0026 - val_loss: 0.0025
Epoch 85/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0028 - val_loss: 0.0023
Epoch 86/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0025
Epoch 87/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 88/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0025 - val_loss: 0.0026
Epoch 89/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0027 - val_loss: 0.0024
Epoch 90/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 91/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0026 - val_loss: 0.0022
Epoch 92/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0025 - val_loss: 0.0021
Epoch 93/300


Epoch 163/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 6.7389e-04
Epoch 164/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 7.3356e-04
Epoch 165/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 5.4529e-04
Epoch 166/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 7.0189e-04
Epoch 167/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 6.8692e-04
Epoch 168/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 6.4773e-04
Epoch 169/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 7.0646e-04
Epoch 170/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 6.1367e-04
Epoch 171/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 5.8842e-04
Epoch 172/300
36/36 [==============================] - 

36/36 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 5.2194e-04
Epoch 242/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 6.7666e-04
Epoch 243/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 5.8274e-04
Epoch 244/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 6.8980e-04
Epoch 245/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 6.4032e-04
Epoch 246/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 6.8308e-04
Epoch 247/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 248/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 7.4413e-04
Epoch 249/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 9.2384e-04
Epoch 250/300
36/36 [==============================] - 0s 1ms/step - loss

36/36 [==============================] - 0s 1ms/step - loss: 0.0023 - val_loss: 0.0041
Epoch 20/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0022 - val_loss: 0.0037
Epoch 21/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0022 - val_loss: 0.0040
Epoch 22/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0021 - val_loss: 0.0041
Epoch 23/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0021 - val_loss: 0.0041
Epoch 24/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0041
Epoch 25/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0040
Epoch 26/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0021 - val_loss: 0.0039
Epoch 27/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0020 - val_loss: 0.0038
Epoch 28/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0042
Epoch 29/300


...

36/36 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0026
Epoch 249/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0027
Epoch 250/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0025
Epoch 251/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0026
Epoch 252/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 0.0026
Epoch 253/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 0.0027
Epoch 254/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 0.0026
Epoch 255/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 0.0026
Epoch 256/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0025
Epoch 257/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0026
Epoc

36/36 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 0.0026
Epoch 30/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0020 - val_loss: 0.0027
Epoch 31/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0020 - val_loss: 0.0027
Epoch 32/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 0.0027
Epoch 33/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0020 - val_loss: 0.0027
Epoch 34/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0019 - val_loss: 0.0027
Epoch 35/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 0.0027
Epoch 36/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0019 - val_loss: 0.0027
Epoch 37/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0020 - val_loss: 0.0027
Epoch 38/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0018 - val_loss: 0.0028
Epoch 39/300


Epoch 111/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0025
Epoch 112/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 0.0025
Epoch 113/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0025
Epoch 114/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0025
Epoch 115/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0024
Epoch 116/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0024
Epoch 117/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0025
Epoch 118/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0024
Epoch 119/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0025
Epoch 120/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0015 - val_los

36/36 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 0.0021
Epoch 193/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0023
Epoch 194/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0023
Epoch 195/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0023
Epoch 196/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0023
Epoch 197/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0023
Epoch 198/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0023
Epoch 199/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 0.0024
Epoch 200/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0024
Epoch 201/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0023
Epoc

36/36 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0022
Epoch 274/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0022
Epoch 275/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 0.0022
Epoch 276/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 0.0022
Epoch 277/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0023
Epoch 278/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0010 - val_loss: 0.0022
Epoch 279/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0024
Epoch 280/300
36/36 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0023
Epoch 281/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 0.0023
Epoch 282/300
36/36 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 0.0022
Epoc

In [10]:
print('\033[1m' + 'Model without Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,0]), "{:.4f}".format(matrix_2[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,1]), "{:.4f}".format(matrix_2[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,2]), "{:.4f}".format(matrix_2[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,3]), "{:.4f}".format(matrix_2[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,8]), "{:.4f}".format(matrix_2[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,9]), "{:.4f}".format(matrix_2[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,10]), "{:.4f}".format(matrix_2[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,11]), "{:.4f}".format(matrix_2[0,27]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,0]), "{:.4f}".format(matrix_3[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,1]), "{:.4f}".format(matrix_3[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,2]), "{:.4f}".format(matrix_3[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,3]), "{:.4f}".format(matrix_3[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,8]), "{:.4f}".format(matrix_3[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,9]), "{:.4f}".format(matrix_3[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,10]), "{:.4f}".format(matrix_3[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,11]), "{:.4f}".format(matrix_3[0,27]))

print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,0]), "{:.4f}".format(matrix_4[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,1]), "{:.4f}".format(matrix_4[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,2]), "{:.4f}".format(matrix_4[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,3]), "{:.4f}".format(matrix_4[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,8]), "{:.4f}".format(matrix_4[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,9]), "{:.4f}".format(matrix_4[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,10]), "{:.4f}".format(matrix_4[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,11]), "{:.4f}".format(matrix_4[0,27]))

Model without Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0477 0.0178
R2 (eV):	 0.8308 0.1318
RAE (eV):	 0.3772 0.1709
RMSE (eV):	 0.0576 0.0211
MAE (nm):	 10.6358 4.0084
R2 (nm):	 0.8218 0.1473
RAE (nm):	 0.3810 0.1759
RMSE (nm):	 12.8307 4.8387
Absorbance FWHM (direct)
MAE (eV): 	 0.0114 0.0028
R2 (eV):	 0.2736 0.2390
RAE (eV):	 0.8080 0.2060
RMSE (eV):	 0.0164 0.0035
MAE (nm):	 2.9670 0.8451
R2 (nm):	 0.3080 0.3886
RAE (nm):	 0.7616 0.1756
RMSE (nm):	 4.0856 1.2634
Absorbance FWHM (cascade)
MAE (eV): 	 0.0108 0.0019
R2 (eV):	 0.3210 0.1593
RAE (eV):	 0.7662 0.1337
RMSE (eV):	 0.0159 0.0031
MAE (nm):	 2.8942 0.7132
R2 (nm):	 0.3668 0.2472
RAE (nm):	 0.7474 0.1490
RMSE (nm):	 3.9637 1.1194


In [11]:
print('\033[1m' + 'Model with Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,4]), "{:.4f}".format(matrix_2[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,5]), "{:.4f}".format(matrix_2[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,6]), "{:.4f}".format(matrix_2[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,7]), "{:.4f}".format(matrix_2[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,12]), "{:.4f}".format(matrix_2[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,13]), "{:.4f}".format(matrix_2[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,14]), "{:.4f}".format(matrix_2[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,15]), "{:.4f}".format(matrix_2[0,31]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,4]), "{:.4f}".format(matrix_3[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,5]), "{:.4f}".format(matrix_3[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,6]), "{:.4f}".format(matrix_3[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,7]), "{:.4f}".format(matrix_3[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,12]), "{:.4f}".format(matrix_3[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,13]), "{:.4f}".format(matrix_3[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,14]), "{:.4f}".format(matrix_3[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,15]), "{:.4f}".format(matrix_3[0,31]))
print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,4]), "{:.4f}".format(matrix_4[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,5]), "{:.4f}".format(matrix_4[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,6]), "{:.4f}".format(matrix_4[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,7]), "{:.4f}".format(matrix_4[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,12]), "{:.4f}".format(matrix_4[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,13]), "{:.4f}".format(matrix_4[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,14]), "{:.4f}".format(matrix_4[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,15]), "{:.4f}".format(matrix_4[0,31]))

Model with Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0474 0.0179
R2 (eV):	 0.8316 0.1323
RAE (eV):	 0.3754 0.1724
RMSE (eV):	 0.0573 0.0212
MAE (nm):	 10.5677 4.0509
R2 (nm):	 0.8223 0.1484
RAE (nm):	 0.3793 0.1778
RMSE (nm):	 12.7655 4.8812
Absorbance FWHM (direct)
MAE (eV): 	 0.0114 0.0029
R2 (eV):	 0.2841 0.2491
RAE (eV):	 0.8014 0.2132
RMSE (eV):	 0.0163 0.0036
MAE (nm):	 2.9246 0.8391
R2 (nm):	 0.3291 0.3520
RAE (nm):	 0.7499 0.1715
RMSE (nm):	 4.0353 1.2257
Absorbance FWHM (cascade)
MAE (eV): 	 0.0108 0.0018
R2 (eV):	 0.3347 0.1522
RAE (eV):	 0.7653 0.1322
RMSE (eV):	 0.0157 0.0030
MAE (nm):	 2.8548 0.6888
R2 (nm):	 0.3804 0.2397
RAE (nm):	 0.7373 0.1401
RMSE (nm):	 3.9196 1.1035


In [12]:
np.savetxt("NN+MMCN_PL_Val_peak.csv", matrix, delimiter=",")
np.savetxt("NN+MMCN_PL_Val_fwhm.csv", matrix_fwhm, delimiter=",")
np.savetxt("NN+MMCN_PL_Val_cascade.csv", matrix_cascade, delimiter=",")